# 02 - RAG con ChromaDB

Este notebook demuestra cómo crear una base de datos vectorial a partir de un PDF
utilizando la clase `VectorStore` y ChromaDB.

## Contenido
1. Configuración del entorno
2. Carga del documento PDF
3. División en chunks
4. Creación de embeddings y base de datos vectorial
5. Búsqueda semántica (ejemplo de uso)

## 1. Configuración del entorno

In [ ]:
import sys
from pathlib import Path

# Agregar el directorio raíz del proyecto al path
PROJECT_ROOT = Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))

# Paths importantes
PDF_PATH = PROJECT_ROOT / "data" / "pdfs" / "4349.pdf"
EMBEDDINGS_PATH = PROJECT_ROOT / "data" / "embeddings"

print(f"📁 Proyecto: {PROJECT_ROOT}")
print(f"📄 PDF: {PDF_PATH}")
print(f"💾 Embeddings: {EMBEDDINGS_PATH}")

In [ ]:
import os
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv(PROJECT_ROOT / ".env")

# Verificar que las API keys estén configuradas
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError(
        "⚠️ OPENAI_API_KEY no está configurada. "
        "Crea un archivo .env en la raíz del proyecto con tu API key."
    )

print("✅ Variables de entorno cargadas correctamente")

## 2. Carga del documento PDF

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Cargar el PDF
loader = PyPDFLoader(str(PDF_PATH))
documents = loader.load()

print(f"📄 Documento cargado: {PDF_PATH.name}")
print(f"📑 Número de páginas: {len(documents)}")
print(f"\n📝 Preview de la primera página:")
print("-" * 50)
print(documents[0].page_content[:500] + "...")

## 3. División en chunks

Dividimos el documento en chunks más pequeños para mejorar la precisión de las búsquedas.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Configurar el text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

# Dividir los documentos
chunks = text_splitter.split_documents(documents)

print(f"✂️ Documento dividido en {len(chunks)} chunks")
print(f"\n📝 Ejemplo de chunk:")
print("-" * 50)
print(chunks[0].page_content)

## 4. Creación de embeddings y base de datos vectorial

Usamos la clase `VectorStore` para crear la base de datos ChromaDB.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from src.utils.vector_store import VectorStore

# Inicializar el modelo de embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"  # Modelo más económico y eficiente
)

# Crear la base de datos vectorial
# Se guardará automáticamente en data/embeddings/climate_docs/
vector_store = VectorStore(
    collection_name="climate_docs",
    embedding_function=embeddings,
)

print(f"🗄️ VectorStore inicializado")
print(f"📂 Ubicación: {vector_store.persist_directory}")

In [ ]:
# Agregar los documentos a la base de datos vectorial
# Esto puede tomar unos minutos dependiendo del tamaño del documento

print("⏳ Agregando documentos a la base de datos vectorial...")
ids = vector_store.add_documents(chunks)

print(f"✅ Se agregaron {len(ids)} chunks a la base de datos")
print(f"📊 Total de documentos en la colección: {vector_store.get_collection_count()}")

## 5. Búsqueda semántica (Ejemplo de uso)

Ahora podemos hacer búsquedas semánticas sobre el contenido del PDF.

In [ ]:
# Ejemplo de búsqueda básica
query = "¿Cuál es el tema principal del documento?"

results = vector_store.similarity_search(query, k=3)

print(f"🔍 Búsqueda: '{query}'")
print(f"\n📋 Se encontraron {len(results)} resultados relevantes:")
print("=" * 60)

for i, doc in enumerate(results, 1):
    print(f"\n--- Resultado {i} ---")
    print(f"📄 Página: {doc.metadata.get('page', 'N/A')}")
    print(f"📝 Contenido:\n{doc.page_content[:400]}...")
    print()

In [ ]:
# Búsqueda con scores de similitud
query = "clima"

results_with_scores = vector_store.similarity_search_with_score(query, k=5)

print(f"🔍 Búsqueda: '{query}'")
print(f"\n📊 Resultados con scores de similitud:")
print("=" * 60)

for i, (doc, score) in enumerate(results_with_scores, 1):
    print(f"\n--- Resultado {i} (Score: {score:.4f}) ---")
    print(f"📄 Página: {doc.metadata.get('page', 'N/A')}")
    print(f"📝 Contenido:\n{doc.page_content[:300]}...")

## 6. Ejemplo de RAG completo

Combinamos la búsqueda vectorial con un LLM para responder preguntas.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Configurar el LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Crear el retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 4})

# Template para RAG
template = """Responde la pregunta basándote únicamente en el siguiente contexto:

{context}

Pregunta: {question}

Respuesta:"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Crear la cadena RAG
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ Cadena RAG configurada correctamente")

In [ ]:
# Hacer una pregunta usando RAG
question = "¿Cuál es el tema principal del documento?"

print(f"❓ Pregunta: {question}")
print("\n" + "=" * 60)
print("\n💬 Respuesta:")

response = rag_chain.invoke(question)
print(response)

In [ ]:
# Celda interactiva para hacer más preguntas
# Modifica la variable 'pregunta' y ejecuta esta celda

pregunta = "Escribe aquí tu pregunta sobre el documento"

print(f"❓ {pregunta}")
print("\n" + "-" * 60)
respuesta = rag_chain.invoke(pregunta)
print(f"\n💬 {respuesta}")

## 7. Cargar base de datos existente

Si ya tienes una base de datos creada, puedes cargarla sin volver a procesar el PDF.

In [ ]:
# Ejemplo: Cargar una base de datos existente
# (No es necesario agregar documentos de nuevo)

from langchain_openai import OpenAIEmbeddings
from src.utils.vector_store import VectorStore

# Inicializar embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Cargar la base de datos existente
existing_store = VectorStore(
    collection_name="climate_docs",
    embedding_function=embeddings,
)

# Verificar contenido
count = existing_store.get_collection_count()
print(f"📊 Documentos en la base de datos: {count}")

# Hacer búsqueda
if count > 0:
    results = existing_store.similarity_search("clima", k=2)
    print(f"\n🔍 Búsqueda de prueba exitosa: {len(results)} resultados")